In [1]:
import datetime
from pyspark.sql.functions import *
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import csv
import numpy as np

In [2]:
spark = SparkSession.builder \
        .master('local') \
        .appName('First Spark app for testing') \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

print('Spark version: ',spark.version)

Spark version:  2.4.0


In [19]:
df = spark.read.csv('/home/peter/Downloads/LastFM/songs1.tsv',header=True, sep=',')

In [4]:
df.createOrReplaceTempView('songs')
df.cache()

DataFrame[userid: string, timestamp: string, musicbrainz-artist-id: string, artist-name: string, musicbrainz-track-id: string, track-name: string]

In [5]:
df2 = df

In [7]:
df2.printSchema()

root
 |-- userid: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- musicbrainz-artist-id: string (nullable = true)
 |-- artist-name: string (nullable = true)
 |-- musicbrainz-track-id: string (nullable = true)
 |-- track-name: string (nullable = true)



In [33]:
df2.write.format("jdbc") \
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName=TestDatabase") \
    .option("dbtable", "SongsTsv") \
    .option("user", "SA") \
    .option("password", "Posztodi4C4").mode('overwrite').save()

In [19]:
jdbcDF = spark.read.format("jdbc") \
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName=TestDatabase") \
    .option("dbtable", "sample_table") \
    .option("user", "SA") \
    .option("password", "Posztodi4C4").load()

In [20]:
jdbcDF.show()

+---+-----------+--------+
| id|insert_date|    name|
+---+-----------+--------+
|  1| 2019-02-01|   Peter|
|  2| 2019-02-02|  Mohali|
|  3| 2019-02-02|  Albert|
|  4| 2019-01-31|Annamari|
|  5| 2019-01-31|  Zsuzsi|
|  6| 2019-02-01|  Dorina|
+---+-----------+--------+



In [ ]:
jdbcDF.write.format("jdbc") \
        .options(
        url ="jdbc:sqlite:/home/peter/.dbeaver4/.metadata/sample-database-sqlite-1/Chinook.db", \
        dbtable="sample_table_from_SQLserver" \
                ).mode('overwrite').save()

In [10]:
df3 = spark.read.format("jdbc") \
        .options(
        url ="jdbc:sqlite:/home/peter/.dbeaver4/.metadata/sample-database-sqlite-1/Chinook.db", \
        dbtable="Artist" \
                ).load()


In [14]:
df3.show(5)

+--------+-----------------+
|ArtistId|             Name|
+--------+-----------------+
|       1|            AC/DC|
|       2|           Accept|
|       3|        Aerosmith|
|       4|Alanis Morissette|
|       5|  Alice In Chains|
+--------+-----------------+
only showing top 5 rows



In [11]:
df3.createOrReplaceTempView('artist')
df3.cache()

DataFrame[ArtistId: int, Name: string]

In [16]:
x = spark.sql('select * from artist')
x.show(5)

+--------+-----------------+
|ArtistId|             Name|
+--------+-----------------+
|       1|            AC/DC|
|       2|           Accept|
|       3|        Aerosmith|
|       4|Alanis Morissette|
|       5|  Alice In Chains|
+--------+-----------------+
only showing top 5 rows



In [17]:
df3.write.format("jdbc") \
    .option("url", "jdbc:sqlserver://localhost:1433;databaseName=TestDatabase") \
    .option("dbtable", "Artist_from_SQLite") \
    .option("user", "SA") \
    .option("password", "Posztodi4C4").mode('overwrite').save()

In [17]:
df.show(10)

+-----------+--------------------+---------------------+-----------+--------------------+--------------------+
|     userid|           timestamp|musicbrainz-artist-id|artist-name|musicbrainz-track-id|          track-name|
+-----------+--------------------+---------------------+-----------+--------------------+--------------------+
|user_000001|2009-05-04T23:08:57Z| f1b1cf71-bd35-4e9...|  Deep Dish|                null|Fuck Me Im Famous...|
|user_000001|2009-05-04T13:54:10Z| a7f7df4a-77d8-4f1...|   坂本龍一|                null|Composition 0919 ...|
|user_000001|2009-05-04T13:52:04Z| a7f7df4a-77d8-4f1...|   坂本龍一|                null|Mc2 (Live_2009_4_15)|
|user_000001|2009-05-04T13:42:52Z| a7f7df4a-77d8-4f1...|   坂本龍一|                null|Hibari (Live_2009...|
|user_000001|2009-05-04T13:42:11Z| a7f7df4a-77d8-4f1...|   坂本龍一|                null|Mc1 (Live_2009_4_15)|
|user_000001|2009-05-04T13:38:31Z| a7f7df4a-77d8-4f1...|   坂本龍一|                null|To Stanford (Live...|
|user_000001|2009-05-

In [8]:
def func(x):
    if x == 'user_000001':
        return 'a'
    else:
        return 'b'

In [9]:
df2 = df.filter(df['userid'] == 'user_000001').groupby(['artist-name',year('timestamp').alias('year')])\
                        .agg( \
                        count('userid').alias('CountRecords') \
                            )


In [10]:
df2.show(5)

+------------------+----+------------+
|       artist-name|year|CountRecords|
+------------------+----+------------+
|       Donald Byrd|2008|          14|
|Suppamicropamshopp|2006|           1|
|          Kawabata|2009|          10|
|    Johnny Bristol|2008|           1|
|             Karma|2007|           3|
+------------------+----+------------+
only showing top 5 rows



In [11]:
gtfive = udf(lambda cn: 'greater' if cn > 5 else 'less')

In [12]:
df2 = df2.withColumn('gtfive',gtfive(df2.CountRecords))

In [13]:
df2.show()

+--------------------+----+------------+-------+
|         artist-name|year|CountRecords| gtfive|
+--------------------+----+------------+-------+
|         Donald Byrd|2008|          14|greater|
|  Suppamicropamshopp|2006|           1|   less|
|            Kawabata|2009|          10|greater|
|      Johnny Bristol|2008|           1|   less|
|               Karma|2007|           3|   less|
|                Tiga|2007|           1|   less|
|              Afta-1|2008|          83|greater|
|              Gaiser|2007|           6|greater|
|    Part-Time Heroes|2008|          11|greater|
|        Neon Heights|2008|           1|   less|
|        Rei Harakami|2006|           4|   less|
|             Orbital|2006|          10|greater|
|     Designed People|2009|          68|greater|
|              Opiate|2009|           1|   less|
|                Fink|2009|           1|   less|
|           Roy Ayers|2008|           1|   less|
|              D*Note|2007|           4|   less|
|The Other People ..

In [14]:
#check pure duplicates, when even the ID is duplicate
print('Count of rows in DF: ',df.count())
print('Count of rows in DF: ',df.distinct().count())

Count of rows in DF:  1000000
Count of rows in DF:  1000000


In [17]:
#check relative duplicates, dupl without ids
df.select([col for col in df.columns if col != 'userid']).distinct().count()

1000000

In [ ]:
#drop relative duplicates
dfx = df.dropDuplicates(subset=[col for col in df.columns if col != 'userid'])
dfx.show(10)

In [20]:
#check for duplicated ids
#the example doesn't have proper id..
df.agg(
        count('userid').alias('count')
        ,countDistinct('userid').alias('distinct_count')
      ).show()

+-------+--------------+
|  count|distinct_count|
+-------+--------------+
|1000000|            41|
+-------+--------------+



In [ ]:
#after dropping the duplicates we can fairly assume that it's only an ID problem
#to fix the IDs:
df.withColumn('new_id', monotonically_increasing_id()).show(10)

In [21]:
df_miss = df
df_miss.rdd.map(
                lambda row : (row['userid'], sum([c == 'None' for c in row]))
                ).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 24.0 failed 1 times, most recent failure: Lost task 0.0 in stage 24.0 (TID 623, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/peter/Spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in main
    process()
  File "/home/peter/Spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 367, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/home/peter/Spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 390, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/peter/Spark/spark-2.4.0-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/util.py", line 99, in wrapper
    return f(*args, **kwarg